In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error

try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    # Fallback to default if seaborn style not available
    plt.rcParams['figure.facecolor'] = 'white'
    plt.rcParams['axes.grid'] = True
    plt.rcParams['grid.alpha'] = 0.3

print("="*80)
print(" Paper-Ready Hybrid Model Evaluation & LSTM Feature Importance")
print("="*80)

hybrid_model_path = '../models/hybrid_prophet_lstm_model.pkl'
data_path = '../data/industrial_load_hourly.parquet'
split_date = pd.to_datetime('2014-01-01 00:00:00')
TIME_STEPS = 168
N_EXPLAIN = 500      
N_REPEATS = 5       
BATCH_SIZE = 128
OUTPUT_DIR = './results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("\n Loading Hybrid Model Components...")
hybrid_components = joblib.load(hybrid_model_path)
m_lstm = hybrid_components['residual_lstm']
m_prophet = hybrid_components['prophet_model']
residual_scaler = hybrid_components['residual_scaler']
print(" Hybrid model loaded successfully.")

print(" Loading Data...")
df_full = pd.read_parquet(data_path).reset_index().rename(columns={'timestamp':'ds','load':'y'})
df_test = df_full[df_full['ds'] >= split_date].copy()
y_true = df_test['y'].values

print(" Generating Prophet baseline forecast...")
prophet_forecast = m_prophet.predict(df_test[['ds']])
df_test['yhat_prophet'] = prophet_forecast['yhat'].values
y_prophet = df_test['yhat_prophet'].values

print(" Generating LSTM residual forecast...")
prophet_full_forecast = m_prophet.predict(df_full[['ds']])
residuals = df_full['y'].values - prophet_full_forecast['yhat'].values
residual_scaled_values = residual_scaler.transform(residuals.reshape(-1, 1))

last_train_residuals = residual_scaled_values[:len(df_full)-len(df_test)][-TIME_STEPS:]
current_batch = last_train_residuals.reshape(1, TIME_STEPS, 1)

test_scaled_residuals = []
for _ in tqdm(range(len(df_test)), desc="Residual forecast"):
    pred = m_lstm.predict(current_batch, verbose=0)
    pred_scaled = pred[0, 0]
    test_scaled_residuals.append(pred_scaled)
    current_batch = np.append(current_batch[:, 1:, :], [[[pred_scaled]]], axis=1)

predicted_residuals = residual_scaler.inverse_transform(np.array(test_scaled_residuals).reshape(-1, 1))
df_test['residual_lstm'] = predicted_residuals.flatten()
df_test['yhat_hybrid'] = df_test['yhat_prophet'] + df_test['residual_lstm']
y_hybrid = df_test['yhat_hybrid'].values
print(" Hybrid forecast completed.")

def mean_absolute_percentage_error(y_true, y_pred):
    epsilon = np.finfo(np.float64).eps
    return np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, epsilon))) * 100

def calculate_metrics(y_true, y_pred, name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f"\n--- {name} ---")
    print(f"RMSE: {rmse:.2f} | MAE: {mae:.2f} | MAPE: {mape:.2f}%")
    return {'RMSE':rmse, 'MAE':mae, 'MAPE':mape}

prophet_metrics = calculate_metrics(y_true, y_prophet, "Prophet Baseline")
hybrid_metrics = calculate_metrics(y_true, y_hybrid, "Hybrid Model")

fig, ax = plt.subplots(figsize=(18, 6))
ax.plot(df_test['ds'], y_true, label='Actual', color='black', linewidth=1.5, alpha=0.8)
ax.plot(df_test['ds'], y_prophet, label='Prophet', color='#FF6B35', linestyle='--', linewidth=1.5)
ax.plot(df_test['ds'], y_hybrid, label='Hybrid', color='#004E89', linewidth=2)
ax.set_xlabel('Time', fontsize=12)
ax.set_ylabel('Load', fontsize=12)
ax.set_title('Hybrid vs Prophet Forecast', fontsize=14, fontweight='bold')
ax.legend(fontsize=11, frameon=True, shadow=True)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'forecast_comparison.png'), dpi=300, bbox_inches='tight')
plt.show()

print("\n Calculating Permutation Importance...")

X_test_sequences = []
test_start_idx = len(df_full) - len(df_test)
for i in range(len(df_test) - TIME_STEPS):
    seq_start = test_start_idx + i
    X_test_sequences.append(residual_scaled_values[seq_start:seq_start + TIME_STEPS])

X_test_scaled = np.array(X_test_sequences).reshape(-1, TIME_STEPS, 1).astype(np.float32)
if len(X_test_scaled) > N_EXPLAIN:
    indices = np.random.choice(len(X_test_scaled), N_EXPLAIN, replace=False)
    X_test_scaled = X_test_scaled[indices]

def permutation_importance(model, X, n_repeats=5):
    baseline_pred = model.predict(X, verbose=0).flatten()
    importances = np.zeros((TIME_STEPS, n_repeats))
    for i in tqdm(range(TIME_STEPS), desc="Permutation"):
        for r in range(n_repeats):
            X_perm = X.copy()
            X_perm[:, i, :] = np.random.permutation(X_perm[:, i, :])
            perm_pred = model.predict(X_perm, verbose=0).flatten()
            importances[i, r] = np.mean(np.abs(perm_pred - baseline_pred))
    return importances

importance_matrix = permutation_importance(m_lstm, X_test_scaled, n_repeats=N_REPEATS)
importance_mean = importance_matrix.mean(axis=1)
importance_std = importance_matrix.std(axis=1)

# Plot Top 20 important time steps
top_idx = np.argsort(-importance_mean)[:20]
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(range(20), importance_mean[top_idx], xerr=importance_std[top_idx], 
               color='#004E89', capsize=3, alpha=0.8, edgecolor='black', linewidth=0.5)
ax.set_yticks(range(20))
ax.set_yticklabels([f"t-{TIME_STEPS-i}" for i in top_idx], fontsize=10)
ax.set_xlabel('Permutation Importance', fontsize=12)
ax.set_title('Top 20 Important Time Steps (LSTM Residual)', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'top20_importance.png'), dpi=300, bbox_inches='tight')
plt.show()

print("\n Numerical Gradient Sensitivity (Finite Differences)...")

def numerical_gradient_sensitivity(model, X, epsilon=1e-4, batch_size=128):
    """
    Calculate sensitivity using finite differences (numerical gradients).
    This avoids issues with custom operations in saved models.
    """
    sens_list = []
    n_samples = min(len(X), 200)
    sample_indices = np.random.choice(len(X), n_samples, replace=False)
    X_sample = X[sample_indices]
    
    for i in tqdm(range(TIME_STEPS), desc="Numerical gradients"):
        sensitivities = []
        for j in range(0, len(X_sample), batch_size):
            batch = X_sample[j:j+batch_size].copy()
            pred_orig = model.predict(batch, verbose=0).flatten()
            batch_perturbed = batch.copy()
            batch_perturbed[:, i, :] += epsilon
            pred_perturbed = model.predict(batch_perturbed, verbose=0).flatten()
            grad = np.abs((pred_perturbed - pred_orig) / epsilon)
            sensitivities.append(np.mean(grad))
        sens_list.append(np.mean(sensitivities))
    
    return np.array(sens_list)

grad_sens = numerical_gradient_sensitivity(m_lstm, X_test_scaled, batch_size=BATCH_SIZE)
grad_sens_norm = (grad_sens - grad_sens.min()) / (grad_sens.max() - grad_sens.min() + 1e-8)
importance_norm = (importance_mean - importance_mean.min()) / (importance_mean.max() - importance_mean.min() + 1e-8)

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(range(TIME_STEPS), importance_norm, label='Permutation Importance', 
        color='#004E89', linewidth=2.5, alpha=0.9)
ax.plot(range(TIME_STEPS), grad_sens_norm, label='Numerical Gradient Sensitivity', 
        color='#FF6B35', alpha=0.7, linewidth=2.5, linestyle='--')
ax.set_xlabel('Time Step Index (t-168 to t-1)', fontsize=12)
ax.set_ylabel('Normalized Importance', fontsize=12)
ax.set_title('Permutation vs Numerical Gradient Sensitivity', fontsize=14, fontweight='bold')
ax.legend(fontsize=11, frameon=True, shadow=True)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'importance_comparison.png'), dpi=300, bbox_inches='tight')
plt.show()

print("\n Creating additional visualizations...")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

prophet_residuals = y_true - y_prophet
axes[0].hist(prophet_residuals, bins=50, color='#FF6B35', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Residual', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Prophet Residuals Distribution', fontsize=12, fontweight='bold')
axes[0].axvline(0, color='black', linestyle='--', linewidth=1)
axes[0].grid(alpha=0.3)

hybrid_residuals = y_true - y_hybrid
axes[1].hist(hybrid_residuals, bins=50, color='#004E89', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Residual', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('Hybrid Residuals Distribution', fontsize=12, fontweight='bold')
axes[1].axvline(0, color='black', linestyle='--', linewidth=1)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'residuals_distribution.png'), dpi=300, bbox_inches='tight')
plt.show()

fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(df_test['ds'], np.abs(prophet_residuals), label='Prophet MAE', 
        color='#FF6B35', alpha=0.6, linewidth=1)
ax.plot(df_test['ds'], np.abs(hybrid_residuals), label='Hybrid MAE', 
        color='#004E89', alpha=0.8, linewidth=1.5)
ax.set_xlabel('Time', fontsize=12)
ax.set_ylabel('Absolute Error', fontsize=12)
ax.set_title('Absolute Error Over Time', fontsize=14, fontweight='bold')
ax.legend(fontsize=11, frameon=True, shadow=True)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'error_over_time.png'), dpi=300, bbox_inches='tight')
plt.show()

results_df = pd.DataFrame({
    'TimeStep': [f"t-{TIME_STEPS-i}" for i in range(TIME_STEPS)],
    'PermutationMean': importance_mean,
    'PermutationStd': importance_std,
    'GradientSensitivity': grad_sens
})
results_df.to_csv(os.path.join(OUTPUT_DIR, 'feature_importance_results.csv'), index=False)

metrics_df = pd.DataFrame({
    'Model': ['Prophet', 'Hybrid'],
    'RMSE': [prophet_metrics['RMSE'], hybrid_metrics['RMSE']],
    'MAE': [prophet_metrics['MAE'], hybrid_metrics['MAE']],
    'MAPE': [prophet_metrics['MAPE'], hybrid_metrics['MAPE']]
})
metrics_df.to_csv(os.path.join(OUTPUT_DIR, 'model_metrics.csv'), index=False)

improvement_df = pd.DataFrame({
    'Metric': ['RMSE', 'MAE', 'MAPE'],
    'Prophet': [prophet_metrics['RMSE'], prophet_metrics['MAE'], prophet_metrics['MAPE']],
    'Hybrid': [hybrid_metrics['RMSE'], hybrid_metrics['MAE'], hybrid_metrics['MAPE']],
    'Improvement (%)': [
        ((prophet_metrics['RMSE'] - hybrid_metrics['RMSE']) / prophet_metrics['RMSE'] * 100),
        ((prophet_metrics['MAE'] - hybrid_metrics['MAE']) / prophet_metrics['MAE'] * 100),
        ((prophet_metrics['MAPE'] - hybrid_metrics['MAPE']) / prophet_metrics['MAPE'] * 100)
    ]
})
improvement_df.to_csv(os.path.join(OUTPUT_DIR, 'model_improvement.csv'), index=False)

print(f"\n All results saved to {OUTPUT_DIR}")
print("\n Model Performance Summary:")
print(metrics_df.to_string(index=False))
print("\n Improvement Summary:")
print(improvement_df.to_string(index=False))
print("\n" + "="*80)
print(" Analysis Complete!")
print("="*80)